In [1]:
from pyspark.ml.feature import VectorAssembler
!pip install pyspark
!pip install findspark

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Analyse search terms on the e-commerce web server").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/30 11:51:06 WARN Utils: Your hostname, BertPro3.local, resolves to a loopback address: 127.0.0.1; using 192.168.68.178 instead (on interface en0)
25/09/30 11:51:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/30 11:51:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv


--2025-09-30 11:51:12--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K   484KB/s    in 0.5s    

2025-09-30 11:51:14 (484 KB/s) - ‘searchterms.csv’ saved [233457/233457]



In [5]:
!pwd

/Users/bert/Project/IBM/Data-Engineering-Course/Data-Engineering-Capstone-Project/06-BigData-with-Spark/tasks


In [6]:
df = spark.read.csv('../data/searchterms.csv', header=True, inferSchema=True)


In [7]:
print(f"Number of Rows is {df.count()} and the number of Columns is {len(df.columns)}")


Number of Rows is 10000 and the number of Columns is 4


In [8]:
df.show(5, truncate=False)


+---+-----+----+--------------+
|day|month|year|searchterm    |
+---+-----+----+--------------+
|12 |11   |2021|mobile 6 inch |
|12 |11   |2021|mobile latest |
|12 |11   |2021|tablet wifi   |
|12 |11   |2021|laptop 14 inch|
|12 |11   |2021|mobile 5g     |
+---+-----+----+--------------+
only showing top 5 rows


In [9]:
df.printSchema()


root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [10]:
df.createOrReplaceTempView('searchterms')
result = spark.sql("select count(*) from searchterms where searchterm = 'gaming laptop'").show()
print(result)

+--------+
|count(1)|
+--------+
|     499|
+--------+

None


In [11]:
result = spark.sql("select searchterm, count(*) as mycount from searchterms group by searchterm order by mycount desc").limit(5).show()
print(result)

+-------------+-------+
|   searchterm|mycount|
+-------------+-------+
|mobile 6 inch|   2312|
|    mobile 5g|   2301|
|mobile latest|   1327|
|       laptop|    935|
|  tablet wifi|    896|
+-------------+-------+

None


In [12]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz


--2025-09-30 11:53:14--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2025-09-30 11:53:15 (203 MB/s) - ‘model.tar.gz’ saved [1490/1490]



In [13]:
!tar -xvzf model.tar.gz


x sales_prediction.model/
x sales_prediction.model/metadata/
x sales_prediction.model/metadata/part-00000
x sales_prediction.model/metadata/.part-00000.crc
x sales_prediction.model/metadata/_SUCCESS
x sales_prediction.model/data/
x sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet
x sales_prediction.model/data/_SUCCESS
x sales_prediction.model/data/.part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet.crc
x sales_prediction.model/data/._SUCCESS.crc: Failed to restore metadata: File exists
tar: Error exit delayed from previous errors.


In [19]:
# from pyspark.mllib.regression import LinearRegressionModel
from pyspark.ml import PipelineModel
from pyspark.ml.regression import LinearRegressionModel


In [18]:
sales_prediction_model = PipelineModel.load("./sales_prediction.model/")

25/09/30 12:09:08 ERROR Instrumentation: java.lang.IllegalArgumentException: requirement failed: Error loading metadata: Expected class name org.apache.spark.ml.PipelineModel but found class name org.apache.spark.ml.regression.LinearRegressionModel
	at scala.Predef$.require(Predef.scala:337)
	at org.apache.spark.ml.util.DefaultParamsReader$.parseMetadata(ReadWrite.scala:715)
	at org.apache.spark.ml.util.DefaultParamsReader$.loadMetadata(ReadWrite.scala:690)
	at org.apache.spark.ml.Pipeline$SharedReadWrite$.$anonfun$load$3(Pipeline.scala:293)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:226)
	at scala.util.Try$.apply(Try.scala:217)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:226)
	at org.apache.spark.ml.Pipeline$SharedReadWrite$.load(Pipeline.scala:292)
	at org.apache.spark.ml.PipelineModel$PipelineModelReader.$anonfun$load$7(Pipeline.scala:381)
	at org.apache.spark.ml.MLEvents.withLoadInstanceEvent(even

IllegalArgumentException: requirement failed: Error loading metadata: Expected class name org.apache.spark.ml.PipelineModel but found class name org.apache.spark.ml.regression.LinearRegressionModel

In [21]:
sales_prediction_model = LinearRegressionModel.load("./sales_prediction.model/")


In [22]:
print(type(sales_prediction_model))

<class 'pyspark.ml.regression.LinearRegressionModel'>


In [24]:
df.printSchema()

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [31]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['year'], outputCol='features')


In [53]:
data = [[2023, 0]]
columns = ["year", "sales"]

In [54]:

dfx = spark.createDataFrame(data, columns)

In [55]:
dfx.printSchema()

root
 |-- year: long (nullable = true)
 |-- sales: long (nullable = true)



In [56]:
dfx.show()

+----+-----+
|year|sales|
+----+-----+
|2023|    0|
+----+-----+



In [57]:

ass = assembler.transform(dfx).select('features', 'sales')

In [58]:
print(ass)

DataFrame[features: vector, sales: bigint]


In [59]:
predictions = sales_prediction_model.transform(ass)
predictions.select('prediction', 'sales').show()

+------------------+-----+
|        prediction|sales|
+------------------+-----+
|175.16564294006457|    0|
+------------------+-----+



25/09/30 13:31:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
